### This script is to write a dashboard for visualizing  precipitation and temperature from the downscaled GCMs with MACA at the Frio and Blanco basins


Note:
  Input: the daily precipitation and temperature at the contributing and recahrge zones at the Frio basin

the csv files are located at ..\dataPre and ..\dataT

### Section 1: Load some general python modoles

In [1]:
# in case the following two modules are not installed
## !pip install ipywidgets
## pip install bqplot

In [2]:
import numpy as np
import random
import pandas as pd
import bqplot.pyplot as plt
from bqplot import *

In [3]:
import os, glob

In [4]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual,Layout,TwoByTwoLayout,GridspecLayout,Box
import datetime

### Section 2: Load the class of data analysis customized for the Frio basin 


The class is included in the python code, dataanalysis.py


In [5]:
from DataAnalysis import Macadata

In [6]:
### get the current file path

filepath=os.getcwd()

In [7]:
filepath

'D:\\downscaled GCMS\\MACA\\4github'

In [8]:
### this file is stored the color names for HTML 
colors=pd.read_csv(os.path.join(filepath,'colors.csv'))

In [9]:
colorline=colors['ColorName'].str.strip().to_list()

### Section 3 Write the dashboard for data visualizion

In [10]:
def initializing_plot():
    
    if rcpwdge.value=='RCP 4.5':
        rcp='rcp45'
    else:
        rcp='rcp85'
    zone=zonewdge.value[:5]
    if Tunitwdge.value=='C':
        itunit=1
    if Tunitwdge.value=='F':
        itunit=2
    if Tunitwdge.value=='K':
        itunit=3
    if Punitwdge.value=='mm':
        ipunit=2
    
    if Punitwdge.value=='inches':
        ipunit=1
    dfT=[]
    dfP=[]
    sel_maca=[]
    sel_basin=[]
    for basin in basinwdge.value:
        objdata=Macadata(basin,filepath,zone,rcp,ipunit,itunit,False)
    
        for macamodel in macamodelwdge.value:
            sel_maca.append(basin)
            sel_basin.append(macamodel)
            if timewdge.value=='Daily':
                dfT.append(objdata.dailyT[macamodel])
                dfP.append(objdata.dailyP[macamodel])
            if timewdge.value=='Monthly':
                dfT.append(objdata.monthlyT[macamodel])
                dfP.append(objdata.monthlyP[macamodel])
            if timewdge.value=='Yearly':
                dfT.append(objdata.annualmeanT[macamodel])
                dfP.append(objdata.annualmeanP[macamodel])

    return dfT,dfP,sel_maca,sel_basin

In [11]:
def response(change):
    
    if rcpwdge.value=='RCP 4.5':
        rcp='rcp45'
    else:
        rcp='rcp85'
    zone=zonewdge.value[:5]
    
    if Tunitwdge.value=='C':
        itunit=1
    if Tunitwdge.value=='F':
        itunit=2
    if Tunitwdge.value=='K':
        itunit=3
    if Punitwdge.value=='mm':
        ipunit=2
    
    if Punitwdge.value=='inches':
        ipunit=1
        
    dfT=[]
    dfP=[]
    sel_maca=[]
    sel_basin=[]
    for basin in basinwdge.value:
        objdata=Macadata(basin,filepath,zone,rcp,ipunit,itunit,False)
    
        for macamodel in macamodelwdge.value:
            sel_maca.append(basin)
            sel_basin.append(macamodel)
            if timewdge.value=='Daily':
                dfT.append(objdata.dailyT[macamodel])
                dfP.append(objdata.dailyP[macamodel])
            if timewdge.value=='Monthly':
                dfT.append(objdata.monthlyT[macamodel])
                dfP.append(objdata.monthlyP[macamodel])
            if timewdge.value=='Yearly':
                dfT.append(objdata.annualmeanT[macamodel])
                dfP.append(objdata.annualmeanP[macamodel])

    new_lines=len(dfP)
    if new_lines>=200:
        raise Exception("Sorry, total numbers of lines over 200!")
    
    for i in range(200):
        
        if i <=new_lines-1:
            basin=sel_basin[i]
            macamodel=sel_maca[i]
            label=[basin+ ': ' + macamodel]
            plotP[i].labels=label
            plotP[i].x=dfP[i].index
            plotP[i].y=dfP[i].values

            plotT[i].labels=label
            plotT[i].x=dfT[i].index
            plotT[i].y=dfT[i].values
        else:
            plotP[i].x=[]
            plotP[i].y=[]
            plotT[i].x=[]
            plotT[i].y=[]

                      
    figP.title=timewdge.value +' Preciptation ' + '('+ Punitwdge.value +')' + ' at the ' + zonewdge.value  
    figT.title=timewdge.value +' Temperature ' + '('+ Tunitwdge.value +')'+ ' at the ' + zonewdge.value 
    pylabel=timewdge.value +' Preciptation ' + '('+ Punitwdge.value +')'
    pax_y.label=pylabel
    tylabel=timewdge.value +' Temperature ' + '('+ Tunitwdge.value +')'
    tax_y.label=tylabel
    #print(len(dfP),len(dfT))
    
def redraw_date_range(date_range):
    
    datemin=date_range[0]
    datemax=date_range[1]
    for i in range(len(plotP)):
        with plotP[i].hold_sync():
            plotP[i].scales['x'].min = datemin
            plotP[i].scales['x'].max = datemax
            
        with plotT[i].hold_sync():
            plotT[i].scales['x'].min = datemin
            plotT[i].scales['x'].max = datemax

In [12]:
descstyle= {'description_width': 'initial'}

basinwdge=widgets.SelectMultiple(
    options=['Frio', 'Blanco'],
    value=['Frio'],
    description='Select basins:',
    style=descstyle
    
)


macamodelwdge=widgets.SelectMultiple(
    options=['bcc-csm1-1-m', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CNRM-CM5',
       'CSIRO-Mk3-6-0', 'GFDL-ESM2G', 'GFDL-ESM2M', 'HadGEM2-CC365',
       'HadGEM2-ES365', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR',
       'IPSL-CM5B-LR', 'MIROC-ESM-CHEM', 'MIROC-ESM', 'MIROC5', 'MRI-CGCM3',
       'NorESM1-M'],
    value=['bcc-csm1-1-m'],
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Select MACA model:',
    style=descstyle,
    
    disabled=False
)

zonewdge=widgets.Dropdown(
    options=['Recharge zone', 'Contributing zone'],
    value='Recharge zone', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Select zone:',
    style=descstyle,
    
    disabled=False
)

rcpwdge=widgets.Dropdown(
    options=['RCP 4.5', 'RCP 8.5'],
    value='RCP 4.5', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Select historical or RCP(4.5 or 8.5):',
    disabled=False,
    style=descstyle,
    
)



timewdge=widgets.Dropdown(
    options=['Daily', 'Monthly', 'Yearly'],
    value='Monthly',
    description='Select time frequency:',
    style=descstyle,
    
)

Tunitwdge=widgets.Dropdown(
    options=['C', 'F','K'],
    value='C', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Select Unit for temperature:',
    disabled=False,
    style=descstyle,
    
)

Punitwdge=widgets.Dropdown(
    options=['mm', 'inches'],
    value='mm', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Select Unit for temperature:',
    disabled=False,
    style=descstyle,
    
)


start_date = datetime.datetime(1950, 1, 1)
end_date = datetime.datetime(2099, 12, 31)

dates = pd.date_range(start_date, end_date, freq='D')
options = [(date.strftime(' %d %b %Y '), date) for date in dates]
index = (0, len(options)-1)
selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    readout=False,
    description='Select date range:',
    style=descstyle
)






# Obtain the data
    
dfT,dfP,sel_maca,sel_basin = initializing_plot()


## Define the plots
fig_margin = dict(top=40, bottom=40, left=60, right=80)

pylabel=timewdge.value +' Preciptation ' + '('+ Punitwdge.value +')'


px_sc = DateScale()
py_sc = LinearScale()
pax_x = Axis(label='Date', scale=px_sc)#, grid_lines='solid', tick_format='0f')
pax_y = Axis(label=pylabel, scale=py_sc, orientation='vertical')#, tick_format='0.2f')

figP = plt.figure( axes=[pax_x, pax_y],
                  title_style={'font-size': '15'},
                  animation_duration=1000,
                  fig_margin=fig_margin)
figP.layout.height = '350px'
plotP=[]
for i in range(len(dfP)):
    ts,basin,macamodel,color=dfP[i],sel_basin[i],sel_maca[i],colorline[i%60]
    label=[basin+ ': ' + macamodel]
    plotP.append(plt.plot(x=ts.index, y=ts.values,scales={'x': px_sc, 'y': py_sc},colors=[color],labels=label,legend_display=True))

for i in range(len(dfP),200):
    color=colorline[i%60]
    plotP.append(plt.plot(x=[], y=[],scales={'x': px_sc, 'y': py_sc},colors=[color],labels=label,legend_display=True ))
    
    
    
figP.title=timewdge.value +' Preciptation ' + '('+ Punitwdge.value +')' + ' at the ' + zonewdge.value 
#plt.legend()

### Prepare for the plot of temperature
tylabel=timewdge.value +' Temperature ' + '('+ Tunitwdge.value +')'

tx_sc = DateScale()
ty_sc = LinearScale()

tax_x = Axis(label='Date', scale=tx_sc)#, grid_lines='solid', tick_format='0f')
tax_y = Axis(label=tylabel, scale=ty_sc, orientation='vertical')#, tick_format='0.2f')

figT = plt.figure(axes=[tax_x, tax_y],
                  title_style={'font-size': '15'},
                  animation_duration=1000,
                  fig_margin=fig_margin)

plotT=[]


for i in range(len(dfT)):
    ts,basin,macamodel,color=dfT[i],sel_basin[i],sel_maca[i],colorline[i%60]
    label=[basin+ ': ' + macamodel]
    plotT.append(plt.plot(x=ts.index, y=ts.values,scales={'x': tx_sc, 'y': ty_sc},colors=[color],labels=label,legend_display=True))
for i in range(len(dfT),200):
    color=colorline[i%60]
    plotT.append(plt.plot(x=[], y=[],scales={'x': tx_sc, 'y': ty_sc},colors=[color],labels=label,legend_display=True ))
    
figT.title=timewdge.value +' Temperature ' + '('+ Tunitwdge.value +')' + \
        ' at the ' + zonewdge.value 

figT.layout.height = '350px'
#plt.legend()



controller = widgets.VBox(children=[basinwdge
                                    ,macamodelwdge
                                    ,zonewdge
                                    ,rcpwdge
                                    ,timewdge
                                    ,Tunitwdge
                                    ,Punitwdge
                                   ])


plotwdge = widgets.VBox(children=[figP,figT])

box_layout = Layout(align_items='center')

grid = GridspecLayout(1, 3,layout=box_layout)#,height='500px')

grid[0,0]=controller
#grid[1,0]=test
grid[0,1:]=plotwdge

    
    
basinwdge.observe(response, names="value")
timewdge.observe(response, names="value")
macamodelwdge.observe(response, names="value")
rcpwdge.observe(response, names="value")
zonewdge.observe(response, names="value")

Tunitwdge.observe(response, names="value")
Punitwdge.observe(response, names="value")


#selection_range_slider.observe(response, names="value")



display(grid)
h=widgets.interact(redraw_date_range,
    date_range=selection_range_slider)
display(h)

GridspecLayout(children=(VBox(children=(SelectMultiple(description='Select basins:', index=(0,), options=('Fri…

interactive(children=(SelectionRangeSlider(description='Select date range:', index=(0, 54786), options=((' 01 …

<function __main__.redraw_date_range(date_range)>